In [1]:
def flow_field_deformation(flow_field, target):
    """
    Given a vector field `flow_field` (H, W, 2) in which every pixel represents the relative 
    displacement vector to its position, then apply the deformation to the target (N, W) 
    image and return the deformed image.
    """
    import SimpleITK as sitk

    target = sitk.GetImageFromArray(target)

    displacement = sitk.DisplacementFieldTransform(2)
    field_size = list(flow_field.shape[:2])
    field_origin = [0, 0]
    field_spacing = [1.0, 1.0]
    field_direction = [1, 0, 0, 1]  # direction cosine matrix (row major order)

    # Concatenate all the information into a single list
    displacement.SetFixedParameters(
        field_size + field_origin + field_spacing + field_direction
    )
    # Set the interpolator to nearest neighbor
    displacement.SetInterpolator(sitk.sitkNearestNeighbor)

    originalDisplacements = flow_field.reshape((-1, 2)).flatten()
    displacement.SetParameters(originalDisplacements)

    resampler = sitk.ResampleImageFilter()
    resampler.SetTransform(displacement)
    resampler.SetReferenceImage(target)
    resampled = resampler.Execute(target)

    return sitk.GetArrayFromImage(resampled)

In [2]:
def check(candidate):
    import numpy as np

    flow_field = np.zeros((5, 5, 2))
    flow_field[3, 2] = [-1, -1]

    image = np.arange(25).reshape((5, 5))
    result = candidate(flow_field, image)
    
    assert result[3, 2] == result[2, 1]

In [3]:
check(flow_field_deformation)